In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import datetime

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit

import gm2
import trfp
import plotting_functions as plt2
# import analysis_helper as helper
# import helper_function_candidates as helper_old
# import df_analysis_funcs as df_func
import helper_functions as helper

import field_map_config_run2 as map_config

from IPython.display import clear_output

blinds = np.loadtxt('blinds.txt')
pair_dict = map_config.pair_dict

open_prefix = 'hdf5/2021-04-26_run_'

save_dir = '/data2/aetb/'
save_prefix = str(datetime.date.today())+'_run-2_'


In [ ]:
runs = map_config.runs

columns = [['st'+str(st)+',dt'] +  ['st'+str(st)+',m'+str(m+1) for m in range(9)] for st in range(72)]
columns = [item for sublist in columns for item in sublist]
sync_offset_df = pd.DataFrame(columns=columns)

for run in runs:
    filename = open_prefix + run + '.h5'

    interp_dfs, keys, subrun_df = helper.read_dfs(filename)
    run_pair_dict = pair_dict[run]

    moment_dfs = helper.interp_to_moment(interp_dfs, keys)
    moment_dfs = helper.blind_moments(moment_dfs, keys, blinds)
    corrected_dfs = helper.moment_to_corrected(moment_dfs, keys)
    hybrid_moment_dfs = helper.bloch_style_moments(corrected_dfs, keys)
    baselines_hybrid = helper.station_average(hybrid_moment_dfs, keys)
    hybrid_vtm_dfs = helper.calculate_vtms(hybrid_moment_dfs, keys, baselines_hybrid, run_pair_dict)
 
    ## Sync Offset
    for pair in pair_dict[run]:
        tr_bl_1 = baselines_hybrid['tr'][pair_dict[run][pair][0]]
        tr_bl_2 = baselines_hybrid['tr'][pair_dict[run][pair][1]]
        fp_bl_1 = baselines_hybrid['fp'][pair_dict[run][pair][0]]
        fp_bl_2 = baselines_hybrid['fp'][pair_dict[run][pair][1]]
        time_bl_1 = baselines_hybrid['time'][pair_dict[run][pair][0]]
        time_bl_2 = baselines_hybrid['time'][pair_dict[run][pair][1]]
        
        delta_tr = tr_bl_2 - tr_bl_1
        delta_fp = fp_bl_2 - fp_bl_1
        delta_time = time_bl_2 - time_bl_1
        
        sync_offsets = np.empty((72,9))
        for st in range(72):
            J = helper._choose_J(st)
            sync_offsets[st,0:5] = delta_tr[st,0:5] -  np.matmul(J, delta_fp[st,0:5])
            sync_offsets[st,5:9] = delta_tr[st,5:9]
        
        row = {}
        for st in range(72):
            for m in range(9):
                row['st'+str(st)+',m'+str(m+1)] = sync_offsets[st,m]
            row['st'+str(st)+',dt'] = delta_time[st]

        sync_offset_df = sync_offset_df.append(pd.DataFrame(row, index=['run_'+run+pair[6]]))
    
    
    ## Done w/ sync offsets

    clear_output()
    
    for key in hybrid_vtm_dfs.keys():
        save_key = 'run_' + run + key[6]
        print 'Saving ' + save_key
        # save hybrid
        save_path = save_dir + save_prefix + 'hybrid_maps.h5'
        hybrid_vtm_dfs[key].to_hdf(save_path, key=save_key)

print 'Saving Sync Offsets'

save_path = save_dir + save_prefix + 'hybrid_syncs.h5'
save_key = 'sync_offset_df'
sync_offset_df.to_hdf(save_path, key=save_key)

print
print 'Done.'

In [ ]:
## need to redo 2i sync offsets for some reason
## 2i has weird negative numbers for baseline
load_path = map_config.path

df1 = helper.root_to_pandas([7842], tr_run=True, prefix=load_path)
df2 = helper.root_to_pandas([7876], tr_run=True, prefix=load_path)
df = helper.root_to_pandas(range(7845, 7873+1), tr_run=False, prefix=load_path)

df1 = helper.trolley_footprint_replacement(helper.calc_moment_df(df1))
df2 = helper.trolley_footprint_replacement(helper.calc_moment_df(df2))
df = helper.calc_moment_df(df)

station_phi = trfp.STATION_BARCODE_PHI
weight10 = (station_phi[2]-station_phi[1])/((station_phi[2]-station_phi[0]))
weight12 = (station_phi[1]-station_phi[0])/((station_phi[2]-station_phi[0]))
weight32 = (station_phi[4]-station_phi[3])/((station_phi[4]-(station_phi[2]-360)))
weight34 = (station_phi[3]-(station_phi[2]-360))/((station_phi[4]-(station_phi[2]-360)))
weight54 = (station_phi[6]-station_phi[5])/((station_phi[6]-station_phi[4]))
weight56 = (station_phi[5]-station_phi[4])/((station_phi[6]-station_phi[4]))

for m in range(1,6):
    df1['st1,m'+str(m)] = weight10*df1['st0,m'+str(m)] + weight12*df1['st2,m'+str(m)]
    df1['st3,m'+str(m)] = weight32*df1['st2,m'+str(m)] + weight34*df1['st4,m'+str(m)]
    df1['st5,m'+str(m)] = weight54*df1['st4,m'+str(m)] + weight56*df1['st6,m'+str(m)]

    df2['st1,m'+str(m)] = weight10*df2['st0,m'+str(m)] + weight12*df2['st2,m'+str(m)]
    df2['st3,m'+str(m)] = weight32*df2['st2,m'+str(m)] + weight34*df2['st4,m'+str(m)]
    df2['st5,m'+str(m)] = weight54*df2['st4,m'+str(m)] + weight56*df2['st6,m'+str(m)]

    df['st1,m'+str(m)] = weight10*df['st0,m'+str(m)] + weight12*df['st2,m'+str(m)]
    df['st3,m'+str(m)] = weight32*df['st2,m'+str(m)] + weight34*df['st4,m'+str(m)]
    df['st5,m'+str(m)] = weight54*df['st4,m'+str(m)] + weight56*df['st6,m'+str(m)]

syncs, times = helper.sync_offset_calc(df1, df2)

row = {}
for st in range(72):
    for m in range(9):
        row['st'+str(st)+',m'+str(m+1)] = syncs[st,m]
    row['st'+str(st)+',dt'] = times[st]

sync_offset_df.loc['run_2i1'] = row

save_path = save_dir + save_prefix + 'hybrid_syncs.h5'
save_key = 'sync_offset_df'
sync_offset_df.to_hdf(save_path, key=save_key)